<a href="https://colab.research.google.com/github/alexeynick/msu_oil_gas/blob/main/module2_lesson1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задачи модуля №2

In [7]:
import pandas as pd

In [8]:
pd.set_option('display.max_colwidth', 100)

## Задача 2.1 Расчет запасов нефти и газа объемным методом

In [19]:
data = {
    'Параметры': ['Площадь нефтеносности залежи',
                  'Эффективная нефтенасыщенная толщина залежи',
                  'Коэффициент пористости',
                  'Коэффициент нефтенасыщенности',
                  'Плотность нефти в поверхностных условиях',
                  'Переводной коэффициент из пластовых условий в поверхностные',
                  'Газовый фактор',
                  'Коэффициент нефтеотдачи',
                  'Добыча нефти с начала разработки на анализируемую дату',
                  ],
        'Единицы измерения': ['м2',
                              'м',
                              'д.ед.',
                              'д.ед.',
                              'т/м3',
                              'д.ед.',
                              'м3/т',
                              'д.ед.',
                              'тыс.т.',
                              ],
         'Значения': [6012,
                     16,
                     0.17,
                     0.515,
                     0.826,
                     0.915,
                     38.7,
                     0.585,
                     2016.9,
                     ]
        }

df = pd.DataFrame(
    data, 
    columns = ['Параметры','Единицы измерения', 'Значения'],
    index=[1, 2, 3, 4, 5, 6, 7, 8, 9])

df

,Параметры,Единицы измерения,Значения
1,Площадь нефтеносности залежи,м2,6012.000
2,Эффективная нефтенасыщенная толщина залежи,м,16.000
3,Коэффициент пористости,д.ед.,0.170
4,Коэффициент нефтенасыщенности,д.ед.,0.515
5,Плотность нефти в поверхностных условиях,т/м3,0.826
6,Переводной коэффициент из пластовых условий в поверхностные,д.ед.,0.915
7,Газовый фактор,м3/т,38.700
8,Коэффициент нефтеотдачи,д.ед.,0.585
9,Добыча нефти с начала разработки на анализируемую дату,тыс.т.,2016.900


$$Q_{ИЗВ} = F * h_{НН} * m * K _{НН} * \rho_{Н} * \beta * \eta $$

где:
- $Q_{ИЗВ}$ - извлекаемые (промышленные) запасы нефти, $т$;
- $F$ — площадь нефтеносности залежи, $м^{2}$;
- $h_{НН}$ — эффективная нефтенасыщенная толщина залежи, $м$;
- $m$ — коэффициент открытой пористости нефтесодержащих пород;
- $K _{НН}$ — коэффициент насыщения пласта нефтью (коэффициент нефтенасыщенности);
- $\rho_{Н}$ — плотность нефти в поверхностных условиях, $т/м^{3}$;
- $\beta$ — пересчетный коэффициент, учитывающий усадку нефти $\beta = \cfrac{1}{b}$ (${b}$ - объемный коэффициент пластовой нефти);
- $\eta$ — коэффициент нефтеотдачи (КИН);


Посчитать:

- 1) Начальные геологические запасы нефти
- 2) Начальные извлекаемые запасы нефти
- 3) Остаточные геологические запасы нефти составят
- 4) Остаточные извлекаемые запасы нефти
- 5) Начальные геологические запасы газа
- 6) Начальные извлекаемые запасы газа
- 7) Остаточные балансовые запасы газа
- 8) Остаточные извлекаемые запасы газа

### Расчеты

$Q_{ИЗВ} = Q_{ГЕОЛ} * \eta $

$Q_{ГЕОЛ} = F * h_{НН} * m * K _{НН} * \rho_{Н} * \beta $

$Q_{ОСТ ГЕОЛ} = Q_{ГЕОЛ} - Q_{ДОБ} $

In [11]:
values = df['Значения']

1    6012.000
2      16.000
3       0.170
4       0.515
5       0.826
6       0.915
7      38.700
8       0.585
9    2016.900
Name: Значения, dtype: float64

In [29]:
F_value = values[1]
hnn_value = values[2]
m_value = values[3]
Knn_value = values[4]
rhon_value = values[5]
beta_value = values[6]
eta_value = values[8]
q_dob = values[9]

q_geol = F_value * hnn_value * m_value * Knn_value * rhon_value * beta_value

In [30]:
print('Начальные геологические запасы нефти: {0:.2f}'.format(q_geol))
print('Начальные извлекаемые запасы нефти: {0:.2f}'.format(q_geol * eta_value))

Начальные геологические запасы нефти: 6364.97
Начальные извлекаемые запасы нефти: 3723.51
